<a href="https://colab.research.google.com/github/mscharan-analytics/GenAI-and-LLMs/blob/main/My_research_Agent_GDPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries

In [ ]:
pip install requests beautifulsoup4

In [ ]:
!pip install -q crewai 'crewai[tools]' python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 35.1 MB/s e

## Import Modules

In [ ]:
import crewai

In [ ]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [ ]:
import os
from crewai import Agent, Task, Process, Crew
from crewai_tools import SerperDevTool

In [ ]:
search_tool =SerperDevTool()

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from langchain.agents import Tool
from bs4 import BeautifulSoup
import requests

## Define Data Extraction Function

## Define Custom Tools

## Define Agents

In [ ]:
researcher = Agent(
    role='Senior Regulations Researcher',
    goal='Conduct research on Regulations that need to be regulated by the government',
    backstory='You are a Reguations expert. You now have to research something that the government should regulate and give context and also the results as to why something needs to be regulated.',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.4)
)

In [ ]:
regulation_advisor = Agent(
    role='Senior Regulations Advisor',
    goal='Provide new Regulations and rules as well as how new regulations should be implemented',
    backstory='You are a regulations advisor. You work with the US government to propose new regulations as well as how they can be enacted',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
)

In [ ]:
gdpr_researcher = Agent(
    role='GDPR Expert',
    goal='Researcher of GDPR guidelines.',
    backstory='You are a General Data Protection Regulation expert. You now have to research something that is provided to be',
    verbose=True,
    allow_delegation=False,
    tools=[search_tool],
    llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.2)
)

In [ ]:
report_writer = Agent(
    role='Report Writer',
    goal='Combine all outputs from previous tasks into a report.',
    backstory='You only combine the information thats it.',
)

## Define Tasks

In [ ]:
def create_tasks():
    task0 = Task(
        description="""What issues should be regulated by the government? Make a list.""",
        expected_output="List of issues that should be regulated by the government",
        agent=researcher
    )

    task1 = Task(
        description="""
        Choose one issue from the list of issues that should be regulated by the government.
        Write a new proposed regulation for this issue. Explain how you would enforce it and what the consequences would be if a company violates the regulation.
        Give points for all the above, key provision within regulation and give multiple points for all.
        Write it in a clear format with headings and bulleted points and explainations.
        """,
        expected_output="Proposed regulation, enforcement strategy, and consequences for violation",
        agent=regulation_advisor
    )

    task2 = Task(
        description="""
        Search the GDPR to see if they have similar regulations to the proposed one.
        Identify which sections of the GDPR are relevant to the proposed regulation.
        Give the articles and the similarities with the proposed regulation
        """,
        expected_output="Comparison of the proposed regulation with the GDPR and relevant sections",
        agent=gdpr_researcher
    )
    task3 = Task(
    description="""
    Combine all the outputs from the previous tasks into a comprehensive report. The report should not omit any information from the previous tasks.
    Make it well structrued output without omitting any information.
    """,
    expected_output="Final report combining task0, task1, task2",
    agent=report_writer,
    dependencies=[task0, task1, task2]
    )

    return [task0, task1, task2, task3]


In [ ]:
def create_tasks():
    task0 = Task(
        description="""Identify issues that should be regulated by the government. Provide a comprehensive list.""",
        expected_output="List of issues that should be regulated by the government",
        agent=researcher
    )

    task1 = Task(
        description="""
        Choose one issue from the list and propose a new regulation for it. Include details on enforcement strategies and consequences for violations.
        """,
        expected_output="Proposed regulation, enforcement strategy, and consequences for violation",
        agent=regulation_advisor
    )

    task2 = Task(
        description="""
        Research the General Data Protection Regulation (GDPR) to find similar regulations. Identify relevant sections of the GDPR applicable to the proposed regulation.
        """,
        expected_output="Comparison of the proposed regulation with the GDPR and relevant sections",
        agent=gdpr_researcher
    )

    return [task0, task1, task2]


 ## Example test

In [ ]:
# Create the tasks
tasks = create_tasks()

# Define the crew with appropriate agents
crew = Crew(
    agents=[researcher, regulation_advisor, gdpr_researcher, report_writer],
    tasks=tasks,
    verbose=1
)

# Start the task execution
result = crew.kickoff()

 [DEBUG]: == Working Agent: Senior Regulations Researcher


> Entering new CrewAgentExecutor chain...
I need to research and identify the issues that should be regulated by the government to ensure the safety and well-being of the public.

Action: Search the internet
Action Input: {"search_query": "issues that should be regulated by the government"} 


Search results: Title: Regulatory Issues & Challenges Facing Businesses in 2024 - Paychex
Link: https://www.paychex.com/articles/compliance/top-regulatory-issues
Snippet: Top Regulatory Issues of 2024: What Businesses Should Know and Prepare For · Artificial Intelligence Regulation · Data Privacy and Cybersecurity.
---
Title: Section 2: Views of Government Regulation | Pew Research Center
Link: https://www.pewresearch.org/politics/2012/02/23/section-2-views-of-government-regulation/
Snippet: In general, more Americans say that government regulation of business is harmful than say it is necessary to protect the public.
---
Title: Governme

In [ ]:
pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.3 MB/s eta 0:00:00


In [ ]:
# Export the final report to PDF
def export_to_pdf(report):
    pdf_filename = "comprehensive_report.pdf"
    doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []

    title = Paragraph("Comprehensive Report", styles["Heading1"])
    elements.append(title)
    elements.append(Spacer(1, 12))

    # Add report content
    content = Paragraph(report, styles["Normal"])
    elements.append(content)

    # Build the PDF
    doc.build(elements)
    print(f"Report exported to {pdf_filename}")

final_report = tasks[-1].output.report
export_to_pdf(final_report)


AttributeError: 'TaskOutput' object has no attribute 'report'

In [ ]:
import random
import numpy as np

# Set a random seed for Python's built-in random module
random.seed(42)

# Set a random seed for numpy
np.random.seed(42)

# Create the tasks
tasks = create_tasks()

# Define the crew with appropriate agents
crew = Crew(
    agents=[researcher, regulation_advisor, gdpr_researcher],
    tasks=tasks,
    verbose=1
)

# Start the task execution
result = crew.kickoff()


 [DEBUG]: == Working Agent: Senior Regulations Researcher


> Entering new CrewAgentExecutor chain...
I need to research and identify a list of issues that should be regulated by the government. I will use the "Search the internet" tool to gather information on this topic.

Action: Search the internet
Action Input: {"issues that should be regulated by the government"}Action: Search the internet
Action Input: {"search_query": "issues that should be regulated by the government"} 


Search results: Title: Regulatory Issues & Challenges Facing Businesses in 2024 - Paychex
Link: https://www.paychex.com/articles/compliance/top-regulatory-issues
Snippet: Top Regulatory Issues of 2024: What Businesses Should Know and Prepare For · Artificial Intelligence Regulation · Data Privacy and Cybersecurity.
---
Title: Section 2: Views of Government Regulation | Pew Research Center
Link: https://www.pewresearch.org/politics/2012/02/23/section-2-views-of-government-regulation/
Snippet: In general, more A

In [ ]:
print('################')
print(result)

################
I apologize for the limitations of the tools provided. Unfortunately, I was unable to retrieve the specific information you requested regarding similar regulations to the GDPR and relevant sections applicable to the proposed Fair Advertising and Product Safety Act. I recommend conducting further research using additional resources or consulting with legal experts familiar with data protection regulations.


In [ ]:
print("Results:")
pprint(result, width=80, indent=2)

Results:
('After researching similar regulations to the proposed legislation on AI, it '
 'is evident that various states in the US have introduced bills and '
 'regulations to govern the development and use of artificial intelligence '
 'technologies. These regulations focus on aspects such as accountability, '
 'transparency, and ethical considerations in AI systems. Additionally, there '
 'is a push for the establishment of regulatory bodies to oversee compliance '
 'and enforcement of AI guidelines.\n'
 '\n'
 'Relevant sections of the GDPR that may apply to the proposed AI regulations '
 'include:\n'
 '\n'
 '- Article 5: Principles relating to processing of personal data\n'
 '- Article 22: Automated individual decision-making, including profiling\n'
 '- Article 35: Data protection impact assessment\n'
 '- Article 37: Designation of the data protection officer\n'
 '- Article 83: General conditions for imposing administrative fines\n'
 '\n'
 'These sections of the GDPR can provide gu